In [1]:
import pandas as pd

df_clean = pd.read_pickle("df_clean_v1_07122021_py35.pkl")
print(df_clean.shape)
df_clean.head()

(16068, 18)


,lyrics,explicit,song_name,song_popularity,mode,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,valence,tempo,release_year,genres,genre,artist
song_id,,,,,,,,,,,,,,,,,,
3e9HZxeyfWwjeyPAMmWSSQ,Thought I'd end up with Sean. But he wasn't a ...,1,"thank u, next",86,1,0.22900,0.717,0.653,0.000000,0.1010,-5.634,0.0658,0.412,106.966,2019,"[dance pop, pop, post-teen pop]",dance/electronic,Ariana Grande
5p7ujcrUXASCNwRaWNHR1C,Found you when your heart was broke. I filled ...,1,Without Me,87,1,0.29700,0.752,0.488,0.000009,0.0936,-7.050,0.0705,0.533,136.041,2018,"[dance pop, electropop, etherpop, indie poptim...",dance/electronic,Halsey
2xLMifQCjDGFmkHkpNLD9h,"Astro, yeah. Sun is down, freezin' cold. That'...",1,SICKO MODE,85,1,0.00513,0.834,0.730,0.000000,0.1240,-3.714,0.2220,0.446,155.008,2018,"[pop, pop rap, rap]",pop rap,Travis Scott
1rqqCSm0Qe4I9rUvWncaom,"High, high hopes. Had to have high, high hopes...",0,High Hopes,86,1,0.19300,0.579,0.904,0.000000,0.0640,-2.729,0.0618,0.681,82.014,2018,"[baroque pop, emo, modern rock, pop punk]",pop,Panic! At The Disco
0bYg9bo50gSsH3LtXe2SQn,I-I-I don't want a lot for Christmas. There is...,0,All I Want for Christmas Is You,63,1,0.16400,0.335,0.625,0.000000,0.0708,-7.462,0.0386,0.346,150.277,1994,18,christmas,Mariah Carey


fine-tuning tasks: https://ruder.io/recent-advances-lm-fine-tuning/
"The pre-trained model is then fine-tuned on labelled data of a downstream task using a standard cross-entropy loss."

Hugging Face examples: https://huggingface.co/docs/transformers/training

# imdb example
https://colab.research.google.com/github/huggingface/notebooks/blob/master/transformers_doc/custom_datasets.ipynb#scrollTo=B0WiFsYNDhbs

# split dataset

In [2]:
# !pip install datasets

In [3]:
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

seed = 11
val_size = 0.15
test_size = 0.15
train, _eval = train_test_split(df_clean, test_size=val_size + test_size, random_state=seed)
val, test = train_test_split(df_clean, test_size=test_size / (val_size + test_size), random_state=seed)
print(train.shape)
print(val.shape)
print(test.shape)


# lyrics_valence = DatasetDict()
# # arguments expected by forward: https://github.com/huggingface/transformers/blob/9aeacb58bab321bc21c24bbdf7a24efdccb1d426/src/transformers/modeling_bert.py#L1313
# lyrics_valence["train"] = Dataset.from_dict({"text": X_train, "labels": y_train})
# lyrics_valence["test"] = Dataset.from_dict({"text": X_test, "labels": y_test})

(11247, 18)
(8034, 18)
(8034, 18)


# prepare for genre classification

In [4]:
labels_lst = list(set(df_clean["genre"]))
print(labels_lst)

from datasets import ClassLabel

labels = ClassLabel(names=labels_lst)

['dance/electronic', 'jazz/blues/funk', 'disco', 'rock', 'christmas', 'soul/motown', 'pop', 'r&b', 'religious', 'pop rap', 'indie/alternative', 'hard rock/metal', 'acoustic/folk', 'country', 'hip-hop/rap', 'classic rock', 'adult standards', 'reggae', 'latin', 'movie']


In [5]:
lyrics_genre = DatasetDict()

# arguments expected by forward: https://github.com/huggingface/transformers/blob/9aeacb58bab321bc21c24bbdf7a24efdccb1d426/src/transformers/modeling_bert.py#L1313
lyrics_genre["train"] = Dataset.from_dict({"text": list(train["lyrics"]), "labels": labels.str2int(train["genre"])})
lyrics_genre["validation"] = Dataset.from_dict({"text": list(val["lyrics"]), "labels": labels.str2int(val["genre"])})
lyrics_genre["test"] = Dataset.from_dict({"text": list(test["lyrics"]), "labels": labels.str2int(test["genre"])})

print(lyrics_genre)

DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 11247
    })
    validation: Dataset({
        features: ['text', 'labels'],
        num_rows: 8034
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 8034
    })
})


In [6]:
# cast to ClassLabel
for _key in lyrics_genre.keys():
    new_features = lyrics_genre[_key].features.copy()
    new_features["labels"] = labels
    lyrics_genre[_key] = lyrics_genre[_key].cast(new_features)
    

# new_features = lyrics_genre["train"].features.copy()
# new_features["label"] = labels

# lyrics_genre["train"] = lyrics_genre["train"].cast(new_features)

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [7]:
lyrics_genre["test"].features

{'text': Value(dtype='string', id=None),
 'labels': ClassLabel(num_classes=20, names=['dance/electronic', 'jazz/blues/funk', 'disco', 'rock', 'christmas', 'soul/motown', 'pop', 'r&b', 'religious', 'pop rap', 'indie/alternative', 'hard rock/metal', 'acoustic/folk', 'country', 'hip-hop/rap', 'classic rock', 'adult standards', 'reggae', 'latin', 'movie'], names_file=None, id=None)}

### preprocess / tokenize

In [8]:
# available models: https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoConfig.from_pretrained
from transformers import AutoTokenizer

# model_str = "gpt2"
# model_str = "bert-base-cased"
model_str = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_str, use_fast=True)
# tokenizer(lyrics_genre["train"][0]["text"])

In [9]:
if model_str == "gpt2":
    tokenizer.pad_token = tokenizer.eos_token  # for gpt2

max_length = 256
    
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)
#     tokens = tokenizer(examples['text'], padding=True, truncation=True, max_length=max_length)
#     return tokens


tokenized_lyrics = lyrics_genre.map(preprocess_function, batched=True)

  0%|          | 0/12 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [10]:
# smaller datasets for iteration
n = 1000
small_train_dataset = tokenized_lyrics["train"].shuffle(seed=seed).select(range(n)) 
small_val_dataset = tokenized_lyrics["validation"].shuffle(seed=seed).select(range(n)) 
small_test_dataset = tokenized_lyrics["test"].shuffle(seed=seed).select(range(n)) 

full_train_dataset = tokenized_lyrics["train"]
full_validation_dataset = tokenized_lyrics["validation"]
full_test_dataset = tokenized_lyrics["test"]

In [11]:
print(small_train_dataset)

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'text'],
    num_rows: 1000
})


# fine-tuning!

In [12]:
# !pip install torch==1.8.1

In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(model_str, num_labels=len(labels_lst))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classi

In [14]:
batch_size = 32
metric_name = "accuracy"
gpu_batch_size = 0

training_args = TrainingArguments(
    f"{model_str}-finetuned-genre", 
    
#     evaluation_strategy="epoch",
#     learning_rate=2e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     per_gpu_train_batch_size=gpu_batch_size,
#     per_gpu_eval_batch_size=gpu_batch_size,
#     num_train_epochs=5,
#     weight_decay=0.01,
    
    evaluation_strategy = "epoch",
    
#     evaluation_strategy="steps",
#     eval_steps=1000,
    
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
#     push_to_hub=True,
)

In [15]:
# # DOESN'T WORK!!!
# from transformers import DataCollatorForTokenClassification

# data_collator = DataCollatorForTokenClassification(tokenizer)

In [16]:
from datasets import load_metric
import numpy as np


TEST = False
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset if TEST else full_train_dataset,
    eval_dataset=small_val_dataset if TEST else full_validation_dataset,
#     data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [17]:
# check data
len(trainer.train_dataset[0]["attention_mask"]) == len(trainer.train_dataset[0]["input_ids"])
print(trainer.train_dataset[0]["labels"])
print(trainer.train_dataset.features["labels"].names)

3
['dance/electronic', 'jazz/blues/funk', 'disco', 'rock', 'christmas', 'soul/motown', 'pop', 'r&b', 'religious', 'pop rap', 'indie/alternative', 'hard rock/metal', 'acoustic/folk', 'country', 'hip-hop/rap', 'classic rock', 'adult standards', 'reggae', 'latin', 'movie']


In [18]:
len(small_train_dataset[0]["input_ids"])

512

In [ ]:
# debugging: https://huggingface.co/course/chapter8/4?fw=pt
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 11247
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1760


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,2.117067,0.344785
2,2.296800,1.923089,0.405527
3,1.976700,1.788807,0.452452
4,1.976700,1.681936,0.482823


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8034
  Batch size = 32
Saving model checkpoint to distilbert-base-uncased-finetuned-genre/checkpoint-352
Configuration saved in distilbert-base-uncased-finetuned-genre/checkpoint-352/config.json
Model weights saved in distilbert-base-uncased-finetuned-genre/checkpoint-352/pytorch_model.bin
tokenizer config file saved in distilbert-base-uncased-finetuned-genre/checkpoint-352/tokenizer_config.json
Special tokens file saved in distilbert-base-uncased-finetuned-genre/checkpoint-352/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 8034
  Batch size = 32
Saving model checkpoint to distilbert-base-uncased-finet

In [23]:
import torch
torch.cuda.empty_cache()

try:
    del full_train_dataset
except:
    pass
try:
    del full_validation_dataset
except:
    pass
try:
    del full_test_dataset
except:
    pass

import gc
gc.collect()

0

# predicting valence


In [2]:
# available models: https://huggingface.co/docs/transformers/model_doc/auto#transformers.AutoConfig.from_pretrained
from transformers import AutoTokenizer

# model_str = "gpt2"
model_str = "bert-base-cased"
model_str = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_str)

In [3]:
tokenizer.model_max_length

512

In [4]:
n = 1000
subset = list(df_clean["lyrics"][:n])
score = list(df_clean["valence"][:n])

In [5]:
# split into train and eval
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(subset, score, test_size=0.3, random_state=11)

In [6]:
from datasets import Dataset, DatasetDict

lyrics_valence = DatasetDict()
# arguments expected by forward: https://github.com/huggingface/transformers/blob/9aeacb58bab321bc21c24bbdf7a24efdccb1d426/src/transformers/modeling_bert.py#L1313
lyrics_valence["train"] = Dataset.from_dict({"text": X_train, "labels": y_train})
lyrics_valence["test"] = Dataset.from_dict({"text": X_test, "labels": y_test})

In [7]:
# https://github.com/huggingface/transformers/issues/2648#issuecomment-616177044
if model_str == "gpt2":
    tokenizer.pad_token = tokenizer.eos_token  # for gpt2


def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

tokenized_lyrics = lyrics_valence.map(preprocess_function, batched=True)

train_dataset = tokenized_lyrics["train"]
eval_dataset = tokenized_lyrics["test"]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [8]:
tokenized_lyrics.shape

{'train': (700, 4), 'test': (300, 4)}

In [9]:
tokenized_lyrics["train"]

Dataset({
    features: ['attention_mask', 'input_ids', 'labels', 'text'],
    num_rows: 700
})

# fine-tuning

In [10]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(model_str, num_labels=1)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [11]:
from transformers import TrainingArguments

batch_size = 4
gpu_batch_size = 0

training_args = TrainingArguments(
    "test_trainer", 
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    per_gpu_train_batch_size=gpu_batch_size,
    per_gpu_eval_batch_size=gpu_batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [12]:
from transformers import Trainer
import numpy as np
from datasets import load_metric


metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

trainer = Trainer(
    model=model, 
    args=training_args, 
    train_dataset=train_dataset, 
    eval_dataset=eval_dataset, 
#     compute_metrics=compute_metrics
)

In [13]:
# for cleanup
import torch
torch.cuda.empty_cache()

import gc
gc.collect()

23

In [14]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 700
  Num Epochs = 5
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 875


Epoch,Training Loss,Validation Loss
1,No log,0.046868
2,No log,0.042713
3,0.048300,0.042184
4,0.048300,0.047476
5,0.048300,0.047576


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 4
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward`

TrainOutput(global_step=875, training_loss=0.037705250331333706, metrics={'train_runtime': 207.4306, 'train_samples_per_second': 16.873, 'train_steps_per_second': 4.218, 'total_flos': 463627627008000.0, 'train_loss': 0.037705250331333706, 'epoch': 5.0})

In [16]:
predictions = trainer.predict(tokenized_lyrics["test"])
true_val = tokenized_lyrics["test"]["labels"]
print("test error", np.mean((np.array(predictions[0]) - true_val) ** 2))

predictions = trainer.predict(tokenized_lyrics["train"])
true_val = tokenized_lyrics["train"]["labels"]
print("train error", np.mean((np.array(predictions[0]) - true_val) ** 2))

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 300
  Batch size = 4
The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 700
  Batch size = 4


test error 0.06298211452908899
train error 0.08451772985381856


In [2]:
import numpy as np

np.sqrt(0.06298211452908899)

0.2509623767202745

# TODO

multi-task with each acoustic feature and genre